In [1]:
from sage.geometry.hyperplane_arrangement.affine_subspace import AffineSubspace
from itertools import combinations
import itertools

#TODO:
#F_weight can be improved by pre computing C

# Basic Implementations


In [2]:
# Helper functions
def F_Weight(v,F, phi = False, C = False): #Usable for all f-metrics in general
    #v must be a vector and F a matrix in the correct field having as rows the spanning family
    #the F_wwight of v is the coset weight of the counter image of v
    if phi == False:
        phi = linear_transformation(F)
    if C == False:
        C = F.kernel()
    
    #counterimage of v
    x = phi.lift(v)
    
    #calculate minimum weight
    m = x.hamming_weight()
    
    for c in C:
        y = x + c
        m_y = y.hamming_weight()
        
        if m_y < m:
            m = m_y
    
    return m

def F_MinimumDistance(Gamma,F): #Usable for all f-metrics in general
    #Gamma must be a linear code, F as above 
    #it's better to calculate the various spans of F and see when an element it's in them
    m = F.nrows()
    phi = linear_transformation(F)
    C = F.kernel()
    
    for x in Gamma: #how do i only check the projective elements? this can be hard to calculate for large codes!
        
        m_x = F_Weight(x,F, phi = phi, C = C)
        print("new x, m_x and m", x, m_x, m) #debug
        if m_x < m and not(x == 0):
            m = m_x
        if m==1:
            return m
        
    return m


def Is_FMDS(Gamma, F):  #For Phase rotation metric, or if u_F is known
    #Gamma must be a linear code, 
   #I assume F is of a Phase metric, u_F distribution otherwise 
    
    if Gamma.dimension() == 0:
        return True
    else:
        bold_N = F.nrows()
        N = F.ncols()
        u=list(range(0,N+1))
        k = Gamma.dimension()
        d = F_MinimumDistance(Gamma, F)
        print(u)
        print("d, N,  u(d-1), k are:",d,N,u[d-1],k)
    
        return k == N - u[d-1]

def hamming_sphere(N, r, q):
    """
    A generator function to iteratively produce vectors of hamming weight r
    
    Parameters:
    - N: The length of the vectors.
    - r: The weight of the vector
    - q: the cardinality of the finite field
    """
    #chose r indices for not null coordinates
    for indices in combinations(range(N), r):
        # Generate all possible values for these positions
        for coordinates in product(range(q-1), repeat=r):
            # Start with a vector of all zeros
            v = [0] * N
            # Update the vector at the specified indices
            for i, index in enumerate(indices):
                v[index] = coordinates[i] + 1  # +1 because range(q-1) starts from 0
            yield vector(GF(q),v)
            
def sphere_F(F,r,q):
    """
    Given the matrix associated to a parent function F over the field q
    returns the set containing the vectors of weight r.
    """
    N = F.ncols()  # parent space dimension

    # Initialize an empty set to store the vectors
    X = set()

    for v in hamming_sphere(N,r,q):
        phi_v = F*v
        phi_v.set_immutable()
        X.add(phi_v)

    return Set(X)

# Dual Codes
In this section we test some possible definitions of dual codes such as the F_dual: taking the orthogonal of the pre image of a code and then taking it's image via a parent function or the H_dual: the usual dual.
In conclusion with neither of these definitions we have some of the proprieties which characterize dual codes in the hamming metric. For example, the F_dual of an FMDS code (Maximum distance separable respect to the $\mathcal{F}$-metric ) is in general not FMDS.



To construct an example:
q >= bold_N=N+1   and large k

Where:
q: is the cardinality of the finite field
bold_N: is the size of the spanning family defining the projective metric
N: is the size of the vector space the projective metric is defined on
k: 

examples tabe    | data        |  F_dual FMDS    | H_dual FMDS    |is the intersection of C and C_ort empty

q, bold_N, N, k = 7, 5, 4, 4   |    False        |    False       |         True

q, bold_N, N, k = 7, 7, 6, 5   |     True        |    False       |         False

q,bold_N, N, k = 8,  8, 7, 6   |     True        |    False       |         False

q,bold_N, N, k =11, 9, 8, 7    |     False       |    False       |         True

q, bold_N, N, k = 11, 10, 9, 7 |     True        |    True        |         True

q, bold_N, N, k = 11, 11, 10, 7|     True        |    False       |         False

q, bold_N, N, k = 11, 11, 10, 8|     True        |    False       |         False

q, bold_N, N, k = 11, 10, 9, 8 |     True        |    True        |         True



q, bold_N, N, k = 13, 9, 8, 7  |     False       |    False       |        True

q, bold_N, N, k = 13, 10, 9, 8 |     False       |    False       |        True

q, bold_N, N, k = 13, 11, 10, 8|     False       |    False       |        True

q, bold_N, N, k = 16, 11, 10, 8|     False       |    False       |        True

q, bold_N, N, k = 16, 12, 11, 9|     False       |    False       |        False




Conjecture:
When q==bold_N the F-dual of FMDS is FMDS







In [1]:
# parameters


q, bold_N, N, k = 7, 5, 4, 4 


Fq.<a> = GF(q, modulus="primitive")


evaluation_points = Fq.list()[:bold_N]; evaluation_points

print(evaluation_points)

# To get an FMDS code from the image of an MDS the following must be true: d_H(D)<=floor(d_H(C)-1/2)
# In such case gamma should always be and FMDS with this program
print(bold_N-k+1 <= floor((bold_N-1)/2),bold_N-k+1,floor((bold_N-1)/2))

[0, 1, 2, 3, 4]
True 2 2


In [2]:
# map phi with parent code as kernel
# check if the intersection is empty

phi_mat = Matrix(Fq, matrix.identity(N).rows() + [-1*ones_matrix(1, N).rows()[0]])
phi = linear_transformation(phi_mat)
C=LinearCode(phi.kernel())
G_C=C.generator_matrix()
G_Cort=C.dual_code().generator_matrix()
print("Is C + C^orth = F_q^(bold N)?",rank(G_C.stack(G_Cort))==bold_N)





Is C + C^orth = F_q^(bold N)? True


In [ ]:
# Reed-Solomon (RS) code

D = codes.GeneralizedReedSolomonCode(evaluation_points, k)
G = D.generator_matrix() 
print(G)

[1 1 1 1 1]
[0 1 2 3 4]
[0 1 4 2 2]
[0 1 1 6 1]


In [ ]:
# phi(RS-code)



gamma = LinearCode(G*phi_mat)
G_gamma = gamma.generator_matrix().echelon_form()
print(G_gamma)



print("Is Gamma FMDS?", Is_FMDS(gamma, phi_mat)) #should be yes


[1 0 0 2]
[0 1 0 6]
[0 0 1 5]
new x, m_x and m (0, 0, 0, 0) 0 5
new x, m_x and m (1, 0, 0, 2) 2 5
new x, m_x and m (2, 0, 0, 4) 2 2
new x, m_x and m (3, 0, 0, 6) 2 2
new x, m_x and m (4, 0, 0, 1) 2 2
new x, m_x and m (5, 0, 0, 3) 2 2
new x, m_x and m (6, 0, 0, 5) 2 2
new x, m_x and m (0, 1, 0, 6) 2 2
new x, m_x and m (1, 1, 0, 1) 2 2
new x, m_x and m (2, 1, 0, 3) 3 2
new x, m_x and m (3, 1, 0, 5) 3 2
new x, m_x and m (4, 1, 0, 0) 2 2
new x, m_x and m (5, 1, 0, 2) 3 2
new x, m_x and m (6, 1, 0, 4) 3 2
new x, m_x and m (0, 2, 0, 5) 2 2
new x, m_x and m (1, 2, 0, 0) 2 2
new x, m_x and m (2, 2, 0, 2) 2 2
new x, m_x and m (3, 2, 0, 4) 3 2
new x, m_x and m (4, 2, 0, 6) 3 2
new x, m_x and m (5, 2, 0, 1) 3 2
new x, m_x and m (6, 2, 0, 3) 3 2
new x, m_x and m (0, 3, 0, 4) 2 2
new x, m_x and m (1, 3, 0, 6) 3 2
new x, m_x and m (2, 3, 0, 1) 3 2
new x, m_x and m (3, 3, 0, 3) 2 2
new x, m_x and m (4, 3, 0, 5) 3 2
new x, m_x and m (5, 3, 0, 0) 2 2
new x, m_x and m (6, 3, 0, 2) 3 2
new x, m_x and m (

In [ ]:
# RS-code dual

D_dual = D.dual_code()
G_dual = D_dual.generator_matrix().echelon_form()
print(G_dual)




[1 3 6 3 1]


In [ ]:
# phi(RS-code dual)

gamma_dual = LinearCode(G_dual*phi_mat)
G_gamma_dual = gamma_dual.generator_matrix().echelon_form()
print(G_gamma_dual)

print("Is the Gamma^d FMDS?", Is_FMDS(gamma_dual, phi_mat))

[0 1 6 1]
new x, m_x and m (0, 0, 0, 0) 0 5
new x, m_x and m (0, 2, 5, 2) 3 5
new x, m_x and m (0, 4, 3, 4) 3 3
new x, m_x and m (0, 6, 1, 6) 3 3
new x, m_x and m (0, 1, 6, 1) 3 3
new x, m_x and m (0, 3, 4, 3) 3 3
new x, m_x and m (0, 5, 2, 5) 3 3
[0, 1, 2, 3, 4]
d, N,  u(d-1), k are: 3 4 2 1
Is the Gamma^d FMDS? False


In [ ]:
# gamma_orthogonal

gamma_orthogonal = gamma.dual_code()
G_orthogonal=gamma_orthogonal.generator_matrix().echelon_form()
print(G_orthogonal)
print(gamma_orthogonal.minimum_distance())
print("Is the Orthogonal of Gamma FMDS?", Is_FMDS(gamma_orthogonal, phi_mat))

[1 3 6 3]
4
new x, m_x and m (0, 0, 0, 0) 0 5
new x, m_x and m (1, 3, 6, 3) 3 5
new x, m_x and m (2, 6, 5, 6) 3 3
new x, m_x and m (3, 2, 4, 2) 3 3
new x, m_x and m (4, 5, 3, 5) 3 3
new x, m_x and m (5, 1, 2, 1) 3 3
new x, m_x and m (6, 4, 1, 4) 3 3
[0, 1, 2, 3, 4]
d, N,  u(d-1), k are: 3 4 2 1
Is the Orthogonal of Gamma FMDS? False


In [ ]:
# MATROID EQUIVALENCE

In [ ]:
#Examples of two matroid equivalent projective metrics with different sphere sizes
N = 4  # parent space dimension
r = 2  # projective sphere radius
q = 5 #cardinality of finite field
# parent function/matrix
F = Matrix(GF(q),[[0, 0, 1], [0, 1, 0], [1, 0, 0], [1, 1, 1],[1,2,2]]).transpose()
G = Matrix(GF(q),[[0, 0, 1], [0, 1, 0], [1, 0, 0], [1, 1, 1],[1,3,2]]).transpose()

B_F = sphere_F(F,r,q)
B_G = sphere_F(G,r,q)
print(B_F.cardinality(), B_G.cardinality())

TypeError: 'int' object is not iterable

In [ ]:
# Example in which anticode bound and mu bound differ.


In [ ]:
q = 2
B = Matrix(GF(q),[[1,1,1,1,0,0,1,0,0,0],[1,1,0,0,1,1,0,1,0,0],[0,0,1,1,1,1,0,0,1,0],[1,1,1,1,1,1,0,0,0,1]]).transpose()
A = Matrix.identity(10)
F = block_matrix(1,2,[B,A])
T = Matrix(GF(q),[[1,1,1,1,0,0,0,0,0,0],[1,1,0,0,1,1,0,0,0,0],[0,0,1,1,1,1,0,0,0,0],[1,1,1,1,1,1,0,0,0,0]]).transpose()
T = LinearCode(T)

phi = linear_transformation(F.transpose())
C=LinearCode(phi.kernel())
C.minimum_distance() # is 6 this implies that it's no generated by elements in G



6

No subset of three in $\mathcal{F}$ is in ball of size 2.
Let $f_1,f_2,f_3 \in \mathcal{F}$. Consider $v :=f_1+f_2+f_3$, this has $\mathcal{F}$-weight two. Thus there exist $f_4,f_5 \in \mathcal{F}$ such that
$v = f_4 + f_5$. At least one between $f_4$ and $f_5$ is different from $f_i,\; i \leq 3$. Thus the minimum distance $\mathcal{C} = ker(F)$ is smaller than $5$ 
and this gives a contraddiction.

In [ ]:
F

[1 1 0 1|1 0 0 0 0 0 0 0 0 0]
[1 1 0 1|0 1 0 0 0 0 0 0 0 0]
[1 0 1 1|0 0 1 0 0 0 0 0 0 0]
[1 0 1 1|0 0 0 1 0 0 0 0 0 0]
[0 1 1 1|0 0 0 0 1 0 0 0 0 0]
[0 1 1 1|0 0 0 0 0 1 0 0 0 0]
[1 0 0 0|0 0 0 0 0 0 1 0 0 0]
[0 1 0 0|0 0 0 0 0 0 0 1 0 0]
[0 0 1 0|0 0 0 0 0 0 0 0 1 0]
[0 0 0 1|0 0 0 0 0 0 0 0 0 1]

# Projective Matroid Extensions

## TODOS
- check linear equivalence


In [7]:
# functions
from itertools import combinations
import sage.matroids.matroid
def generate_unique_vectors_iterator(q,L):
    if q < L:
        raise ValueError("q must be at least L to generate vectors of length 6 with unique coordinates.")
    
    # Generate all combinations of 6 distinct integers from the range [2, q-1]
    for comb in combinations(range(1, q+1), L):
        if 1 in comb:
            # Sort each combination to get a unique representation
            yield tuple(sorted(comb))

def hyperplanes(matroid):
    """
    Given a matroid `matroid`, return all linearly independent sets of size `rank - 1`.

    :param matroid: A matroid object in Sage
    :return: A list of independent sets of size `rank - 1`
    """
    # Get the rank of the matroid
    rank = matroid.rank()
    
    # Retrieve all independent sets
    all_independent_sets = matroid.independent_sets()
    
    # Filter for independent sets of size rank - 1
    independent_sets_size_n_minus_1 = [
        s for s in all_independent_sets if len(s) == rank - 1
    ]
    
    return independent_sets_size_n_minus_1



def get_hyperplane_vector(HM):
    ker = HM.kernel()
    hyperplane_vectors = ker.basis()
    return hyperplane_vectors

def normalize(vector):
    Len = len(vector)
    for i in range(Len):
        if vector[i] != 0:
            return vector / vector[i]
    return vector


# Choose all subsets of columns of size n-1

def extended_family(A):
    AM = Matroid(A)
    #calculate sets of n-1 indipendent vecotrs of the matroid
    hps = hyperplanes(AM)
    extended_family = []
    normal_vectors_list = []
    print("calculating hyperplane defining vectors...")
    for i in range(len(hps)):
        vector = normalize(get_hyperplane_vector(A[:,list(hps[i])])[0])
        if vector not in normal_vectors_list:
            normal_vectors_list.append(vector)
    normal_vectors_matrix = Matrix(GF(q), normal_vectors_list).transpose()
    normal_vectors_matroid = Matroid(normal_vectors_matrix)
    print("calculating sets of n-1 independent hyperplanes...")
    # for any set of n-1 indipendent hyperplanes, calculate intersaction and add it to extended family
    hps_normal_vectors = hyperplanes(normal_vectors_matroid)
    print("calculating intersections...")
    extended_family = []
    n_iter = len(hps_normal_vectors) 
    perc = n_iter//100
    for i in range(n_iter):
        if i%perc == 0:
            print(f"{i/n_iter * 100}%")
        vector = normalize(normal_vectors_matrix.transpose()[list(hps_normal_vectors[i])].transpose().kernel().basis()[0])
        if vector not in extended_family:

            extended_family.append(vector)

    A_ext = Matrix(GF(q), extended_family).transpose()
    A_ext_M = Matroid(A_ext)
    return A_ext, A_ext_M



In [11]:


q = 13
phase = A = Matrix(GF(q),[[1,0,0,0,0,0],[0,1,0,0,0,0],[0,0,1,0,0,0],[0,0,0,1,0,0],[0,0,0,0,1,0],[0,1,0,0,0,0],[0,0,0,0,0,1],[1,1,1,1,1,1]]).transpose()#
A = Matrix(GF(q),[[1,0,0,0,0,0],[0,1,0,0,0,0],[0,0,1,0,0,0],[0,0,0,1,0,0],[0,0,0,0,1,0],[0,1,0,0,0,0],[0,0,0,0,0,1],[1,1,1,1,1,1],[1,2,3,4,5,6]]).transpose()
L0 = [[1,0,0,0,0,0],[0,1,0,0,0,0],[0,0,1,0,0,0],[0,0,0,1,0,0],[0,0,0,0,1,0],[0,1,0,0,0,0],[0,0,0,0,0,1],[1,1,1,1,1,1]]
# Example usage

#Create new projective metrix changing last vector

vector_iterator = generate_unique_vectors_iterator(q,6)
next(vector_iterator) #skip first vector
new_vector = next(vector_iterator)
L0.append(new_vector)

B = Matrix(GF(q),L0).transpose()

A_ext, A_ext_M = extended_family(A)

#penso che le cose create in questo modo non siano linearmente equivalenti 
# they are most likely not linear equivalent because f defined by setting the first n+1 vectors.
# they are matroid equivalent because the linear indipendence relation are the same (necessari condition for extendend equivalence)



calculating hyperplane defining vectors...
calculating sets of n-1 independent hyperplanes...
calculating intersections...
0.0%
0.9999785871822873%


KeyboardInterrupt: 

In [ ]:
B_ext, B_ext_M = extended_family(B)

1 x 63 dense matrix over Finite Field of size 13 (use the '.str()' method to see the entries)

In [ ]:
1+1